In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import pandas as pd



In [2]:
df_nf_unique = pd.read_csv('../df_nf_unique.csv')
df_nf_unique

/tmp/ipykernel_2387353/418950838.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nf_unique = pd.read_csv('../df_nf_unique.csv')


,ID,MODELO,SERIE,NUMERO_NOTA_FISCAL,DATA_EMISSAO,CNPJ_CPF_EMITENTE,RAZAO_SOCIAL_EMITENTE,NOME_FANTASIA_EMITENTE,TIPO_DESTINATARIO,CNPJ_CPF_DESTINATARIO,...,DESCRICAO_PRODUTO,UNIDADE_PRODUTO,UNIDADE_PRODUTO_TRIBUTAVEL,QUANTIDADE_PRODUTO,QUANTIDADE_PRODUTO_TRIBUTAVEL,VALOR_PRODUTO,VALOR_PRODUTO_TRIBUTAVEL,CODIGO_BARRA,CODIGO_BARRA_TRIBUTAVEL,NCM
0,10,55,21,37005,2022-01-01 00:00:00.000,56228356013381,CRBS S/A - CDC Rio Branco,CDC RIO BRANCO,PESSOA_JURIDICA,NaN,...,"SKOL 600ML 6,...",Un,Un,2.0,2.0,5.69,5.69,7891149200405,7891149200405,22030000
1,14,55,21,37005,2022-01-01 00:00:00.000,56228356013381,CRBS S/A - CDC Rio Branco,CDC RIO BRANCO,PESSOA_JURIDICA,NaN,...,"ANTARCTICA PILSEN LATA 350ML SH C/12 NPAL 10,...",Un,Un,12.0,12.0,10.22,10.22,7891991002141,7891991000796,22030000
2,16,55,21,37005,2022-01-01 00:00:00.000,56228356013381,CRBS S/A - CDC Rio Branco,CDC RIO BRANCO,PESSOA_JURIDICA,NaN,...,"PEPSI COLA PET 2L CAIXA C/6 9,...",Un,Un,1.0,1.0,9.61,9.61,7892840800246,7892840800000,22021000
3,53,55,1,2148830,2022-01-01 00:00:00.000,3007331001032,EBAZAR.COM.BR. LTDA,NaN,PESSOA_FISICA,NaN,...,Potes Hermeticos Porta-Mantimentos Tampa Bambu...,UN,UN,1.0,1.0,139.90,139.90,7909569365786,7909569365786,70134290
4,88,55,1,25085,2022-01-01 00:00:00.000,21393527000101,RECOL FARMA LTDA.,RECOL FARMA - 24HORAS,PESSOA_JURIDICA,NaN,...,GRIPINEW C/20 CPR,UN1,UN1,1.0,1.0,8.99,8.99,7896862920029,7896862920029,30045090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2248616,75679419,55,10,12741,2023-07-31 00:00:00.000,5915706000105,COMERCIAL JAPIIM LTDA,COMERCIAL JAPIIM,PESSOA_JURIDICA,NaN,...,ROSQUINHA CHOCOLATE ZERO LOW 150G,UN1,UN1,1.0,1.0,8.07,8.07,7896292070080,7896292070080,19053100
2248617,75679436,55,10,12741,2023-07-31 00:00:00.000,5915706000105,COMERCIAL JAPIIM LTDA,COMERCIAL JAPIIM,PESSOA_JURIDICA,NaN,...,BISC HOT CRACK QUEIJO 150G,UN1,UN1,8.0,8.0,6.26,6.26,7896011105499,7896011105499,19053100
2248618,75679441,55,10,12741,2023-07-31 00:00:00.000,5915706000105,COMERCIAL JAPIIM LTDA,COMERCIAL JAPIIM,PESSOA_JURIDICA,NaN,...,MACARRAO NINHO LARGO PARATI 500G,UN1,UN1,1.0,1.0,5.25,5.25,7896011107028,7896011107028,19021100
2248619,75679480,55,10,12742,2023-07-31 00:00:00.000,5915706000105,COMERCIAL JAPIIM LTDA,COMERCIAL JAPIIM,PESSOA_JURIDICA,NaN,...,GRAFITE ESTAMPADAS,UN1,UN1,1.0,1.0,1.00,1.00,6944504411457,6944504411457,96084000


In [3]:
df_nf_unique.isnull().sum()

ID                                     0
MODELO                                 0
SERIE                                  0
NUMERO_NOTA_FISCAL                     0
DATA_EMISSAO                           0
CNPJ_CPF_EMITENTE                      0
RAZAO_SOCIAL_EMITENTE                  0
NOME_FANTASIA_EMITENTE            294074
TIPO_DESTINATARIO                      0
CNPJ_CPF_DESTINATARIO            2234472
CODIGO_MUNICIPIO_DESTINO               0
NUMERO_PRODUTO                         0
DESCRICAO_PRODUTO                      0
UNIDADE_PRODUTO                       26
UNIDADE_PRODUTO_TRIBUTAVEL             7
QUANTIDADE_PRODUTO                     0
QUANTIDADE_PRODUTO_TRIBUTAVEL          0
VALOR_PRODUTO                          0
VALOR_PRODUTO_TRIBUTAVEL               0
CODIGO_BARRA                           0
CODIGO_BARRA_TRIBUTAVEL                6
NCM                                    0
dtype: int64

In [4]:
es = Elasticsearch(
    "http://localhost:9202",
    basic_auth=('elastic', 'teste123'),
    request_timeout=999999999,
)

es.info()

ObjectApiResponse({'name': 'es01', 'cluster_name': 'es-docker-cluster', 'cluster_uuid': '8pFe7X9oRge9WST4LcJmzw', 'version': {'number': '8.6.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2d58d0f136141f03239816a4e360a8d17b6d8f29', 'build_date': '2023-02-13T09:35:20.314882762Z', 'build_snapshot': False, 'lucene_version': '9.4.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [5]:
indices = es.cat.indices(h='index', format='json')
nomes_indices = [indice['index'] for indice in indices]
nomes_indices

[]

In [6]:
codigos_repetidos = df_nf_unique[df_nf_unique['CODIGO_BARRA_TRIBUTAVEL'] == 'SEM GTIN']
codigos_repetidos

,ID,MODELO,SERIE,NUMERO_NOTA_FISCAL,DATA_EMISSAO,CNPJ_CPF_EMITENTE,RAZAO_SOCIAL_EMITENTE,NOME_FANTASIA_EMITENTE,TIPO_DESTINATARIO,CNPJ_CPF_DESTINATARIO,...,DESCRICAO_PRODUTO,UNIDADE_PRODUTO,UNIDADE_PRODUTO_TRIBUTAVEL,QUANTIDADE_PRODUTO,QUANTIDADE_PRODUTO_TRIBUTAVEL,VALOR_PRODUTO,VALOR_PRODUTO_TRIBUTAVEL,CODIGO_BARRA,CODIGO_BARRA_TRIBUTAVEL,NCM
62648,3100211,55,1,5289,2022-01-29 00:00:00.000,10373001000135,SUPERMERCADO TCHE LTDA ME,SUPERMERCADO TCHE,PESSOA_JURIDICA,NaN,...,COLHER DESC REFEIC SERT PLAST BCA 50UN,UN,UN,20.00,20.00,3.89,3.89,7898032372119,SEM GTIN,39241000
223078,21004643,55,3,1860,2022-06-21 00:00:00.000,529581000153,AUTO POSTO AMAPA EIRELI,AUTO POSTO AMAPA EIRELI,PESSOA_FISICA,NaN,...,DIESEL S-10 ADIT,LT,LT,262.86,262.86,5.39,5.39,7899050521244,SEM GTIN,27101921
224404,21113678,55,1,2106,2022-06-22 00:00:00.000,36158810000118,AUTO POSTO VIA VERDE LTDA,AUTO POSTO VIA VERDE,PESSOA_JURIDICA,NaN,...,DIESEL S-500 ADITIVADO,LTS,LTS,43.23,43.23,6.03,6.03,3000001900207,SEM GTIN,27101921
225311,21164935,55,0,116166,2022-06-22 00:00:00.000,13567147000100,S E G PETROLEO LTDA,AUTO POSTO CRUZEIRO,PESSOA_JURIDICA,NaN,...,DIESEL BS 500 ADITIVADO,L,L,60.70,60.70,8.95,8.95,3000001900207,SEM GTIN,27101259
231229,21523972,55,1,4132,2022-06-24 00:00:00.000,13567147000290,S & G PETROLEO LTDA,S & G PETROLEO LTDA,PESSOA_JURIDICA,NaN,...,DIESEL S500 ADITIVADO,L,L,200.02,200.02,8.95,8.95,3000001900207,SEM GTIN,27101921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2165822,73359493,55,1,38830,2023-07-15 00:00:00.000,17337055000194,POSTO UNIAO DE LIMEIRA LTDA,POSTO UNIAO DE LIMEIRA,PESSOA_JURIDICA,NaN,...,5-OLEO DIESEL B S500 COMUM,L,L,201.60,201.60,4.59,4.59,3000001900207,SEM GTIN,27101921
2206872,74539906,55,1,117436,2023-07-24 00:00:00.000,548431000197,RADIN II LTDA,GAUCHAO 2,PESSOA_JURIDICA,NaN,...,Diesel B S10,l,L,893.52,893.52,5.25,5.25,7899050521244,SEM GTIN,27101921
2218539,74854269,55,1,7632,2023-07-26 00:00:00.000,376437000558,AUTO POSTO ABEL GALINHA LTDA 5,NaN,PESSOA_JURIDICA,NaN,...,OLEO DIESEL B S10 COMUM,LITRO,UNID,70.00,70.00,5.33,5.33,7899050521244,SEM GTIN,27101921
2239004,75426081,55,5,54034,2023-07-31 00:00:00.000,4313357000180,POSTO PAINEIRAO II LTDA,PAINEIRAO II,PESSOA_JURIDICA,NaN,...,DIESEL S10 ADITIVADO IPIMAX,L,L,70.00,70.00,5.09,5.09,7899050521244,SEM GTIN,27101921


In [6]:
!pip3 install -U sentence-transformers
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter nbextension install --py widgetsnbextension --sys-prefix
!jupyter labextension install @jupyter-widgets/jupyterlab-manager


Enabling notebook extension jupyter-js-widgets/extension...
Traceback (most recent call last):
  File "/usr/bin/jupyter-nbextension", line 11, in <module>
    load_entry_point('notebook==6.0.3', 'console_scripts', 'jupyter-nbextension')()
  File "/home/melqui/.local/lib/python3.8/site-packages/jupyter_core/application.py", line 283, in launch_instance
    super().launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/traitlets/config/application.py", line 1043, in launch_instance
    app.start()
  File "/usr/lib/python3/dist-packages/notebook/nbextensions.py", line 983, in start
    super(NBExtensionApp, self).start()
  File "/home/melqui/.local/lib/python3.8/site-packages/jupyter_core/application.py", line 270, in start
    self.subapp.start()
  File "/usr/lib/python3/dist-packages/notebook/nbextensions.py", line 891, in start
    self.toggle_nbextension_python(self.extra_args[0])
  File "/usr/lib/python3/dist-packages/notebook/nbextensions.py", line 864, 

In [7]:
from sentence_transformers import SentenceTransformer
modelL6V2 = SentenceTransformer('all-MiniLM-L6-v2')
modelBV2 = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

modelMult = SentenceTransformer('sentence-transformers/quora-distilbert-multilingual')
modelLASE = SentenceTransformer('sentence-transformers/LaBSE')

/home/melqui/miniconda3/envs/jupyter-env/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/melqui/miniconda3/envs/jupyter-env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/melqui/miniconda3/envs/jupyter-env/lib/python3.12/site-packages/sentence_transformers/models/Dense.py:89: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is

In [11]:
embeddingsL6V2 = modelL6V2.encode(df_nf_unique['DESCRICAO_PRODUTO'].tolist(), show_progress_bar=True)
embeddingsBV2 = modelBV2.encode(df_nf_unique['DESCRICAO_PRODUTO'].tolist(), show_progress_bar=True)
embeddingsMult = modelMult.encode(df_nf_unique['DESCRICAO_PRODUTO'].tolist(), show_progress_bar=True)
embeddingsLASE = modelLASE.encode(df_nf_unique['DESCRICAO_PRODUTO'].tolist(), show_progress_bar=True)
# df_nf_unique['ALL_MINI_BASE_VECTOR'] = df_nf_unique['DESCRICAO_PRODUTO'].apply(lambda x: modelL6V2.encode(x))
# print('sucesso_All_m')
# df_nf_unique['MPNET_BASE_VECTOR'] = df_nf_unique['DESCRICAO_PRODUTO'].apply(lambda x: modelMult.encode(x))
# print('sucesso_mpne')
# df_nf_unique['BV2_BASE_VECTOR'] = df_nf_unique['DESCRICAO_PRODUTO'].apply(lambda x: modelLASE.encode(x))
# print('sucesso_bv2')
# df_nf_unique['LABASE_BASE_VECTOR'] = df_nf_unique['DESCRICAO_PRODUTO'].apply(lambda x: modelLASE.encode(x))
# print('sucesso_labase')




Batches:   0%|          | 0/70270 [00:00<?, ?it/s]

Batches:   0%|          | 0/70270 [00:00<?, ?it/s]

Batches:   0%|          | 0/70270 [00:00<?, ?it/s]

: 

In [ ]:
embeddingsL6V2[0]
embeddingsBV2[0]
embeddingsMult[0]
embeddingsLASE[2248620]


array([-6.43898547e-02, -4.96408902e-03, -7.61796683e-02, -2.18480118e-02,
        3.97624858e-02, -6.33241655e-03,  3.30779282e-03,  1.08664297e-01,
       -4.14992636e-03, -1.05544075e-01,  2.41321027e-02, -5.73128611e-02,
       -4.32627536e-02, -3.45954709e-02, -1.04343586e-01, -3.65461558e-02,
       -9.00622681e-02, -1.21574625e-02, -5.01272678e-02,  5.30551262e-02,
        6.69303983e-02,  7.43479235e-03, -1.50329759e-02,  8.71481672e-02,
       -3.70519352e-03,  3.55149396e-02, -1.48488926e-02, -6.63021172e-04,
       -9.03559104e-03, -6.35159016e-02,  5.99192791e-02,  7.05922171e-02,
        2.09786054e-02, -6.59483522e-02,  1.26359954e-01, -5.83944935e-03,
       -4.50304002e-02,  9.68519226e-03,  5.75246699e-02,  4.61314479e-03,
        4.62166220e-02, -8.35231841e-02,  4.23217118e-02, -8.67899880e-03,
       -3.44813243e-02,  3.77474166e-02, -5.72901918e-03,  6.65366426e-02,
        3.11979204e-02,  7.59749711e-02, -5.79474568e-02, -6.04583062e-02,
       -5.24301827e-02,  

In [32]:
def doc_generator(index_name, df):
  for idx, row in df.iterrows():
    yield {
        "_index": index_name,
        "_id": idx,
        "_source": {
            "modelo": row['MODELO'],
            "serie": row['SERIE'],
            "numero_nota_fiscal": row['NUMERO_NOTA_FISCAL'],
            "data_emissao": row['DATA_EMISSAO'],
            "cnpj_cpf_emitente": row['CNPJ_CPF_EMITENTE'],
            "razao_social_emitente": row['RAZAO_SOCIAL_EMITENTE'],
            "tipo_destinatario": row['TIPO_DESTINATARIO'],
            "valor_produto": row['VALOR_PRODUTO'],
            "codigo_barra": row['CODIGO_BARRA'],
            "ncm": row['NCM'],
            "descricao_produto": row['DESCRICAO_PRODUTO'],
            "all_mini_base_vector": embeddingsL6V2[idx],
            "bv2_base_vector": embeddingsBV2[idx],
            "mpnet_base_vector": embeddingsMult[idx],
            "labase_base_vector": embeddingsLASE[idx]
        }
    }


# def doc_generator(index_name, df):
#   for idx, row in df.iterrows():
#     yield {
#         "_index": index_name,
#         "_id": idx,
#         "_source": {
#             "nome": row['MODELO'],
#             "descricao": row['SERIE'],
#             "vetor_denso_BV2": embeddingsBV2[idx],
#         }
#     }


    

In [34]:
INDEX_NAME = "index_smn_nf"
documentos_nf_index_generator = doc_generator(INDEX_NAME, df_nf_unique)

In [35]:
es.options(ignore_status=[400,404]).indices.delete(index=INDEX_NAME)

body = {
    "settings": {
        'number_of_shards': 3,
        'number_of_replicas': 1,
        "analysis": {
            "analyzer": {
                "standard_asciifolding": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [ "lowercase", "asciifolding" ]
                }
            }
        }
    },
    
    'mappings': {
        'properties': {
            'modelo': {'type': 'integer' },
            'serie': {'type': 'integer' },
            'numero_nota_fiscal': {'type': 'long' },
            'data_emissao': {'type': 'date', 'format': 'yyyy-MM-dd HH:mm:ss.SSS' },
            'cnpj_cpf_emitente': {'type': 'text' },
            'razao_social_emitente': {'type': 'text' },
            'tipo_destinatario': {'type': 'text' },
            'valor_produto': {'type': 'float' },
            'codigo_barra': {'type': 'long' },
            'ncm': {'type': 'long' },
            'descricao_produto': {'type': 'text'},
            'all_mini_base_vector': {
                'type': 'dense_vector',
                'dims': 384,
                'index': True,
                'similarity': 'cosine'
            },
            'bv2_base_vector': {
                'type': 'dense_vector',
                'dims': 768,
                'index': True,
                'similarity': 'cosine'
            },
            'mpnet_base_vector': {
                'type': 'dense_vector',
                'dims': 768,
                'index': True,
                'similarity': 'cosine'
            },
            'labase_base_vector': {
                'type': 'dense_vector',
                'dims': 768,
                'index': True,
                'similarity': 'cosine'
            }
        }
    }

}

es.indices.create(index=INDEX_NAME, body=body)


# es.options(ignore_status=[400,404]).indices.delete(index=INDEX_NAME)

# body = {
#     "settings": {
#         'number_of_shards': 3,
#         'number_of_replicas': 1,
#         "analysis": {
#             "analyzer": {
#                 "standard_asciifolding": {
#                     "type": "custom",
#                     "tokenizer": "standard",
#                     "filter": [ "lowercase", "asciifolding" ]
#                 }
#             }
#         }
#     },
    
#     'mappings': {
#         'properties': {
#             'nome': {'type': 'text' },
#             'descricao': {'type': 'text' },
#             'vetor_denso_BV2': {
#                 'type': 'dense_vector',
#                 'dims': 768,
#                 'index': True,
#                 'similarity': 'cosine'
#             }
#         }
#     }

# }

# es.indices.create(index=INDEX_NAME, body=body)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'index_smn_nf'})

In [36]:
# Verifique o tamanho de um vetor de embedding para cada modelo
print(f"Dimensões do vetor: {len(embeddingsL6V2[0])}")


Dimensões do vetor: 384


In [37]:
indices = es.cat.indices(h='index', format='json')
nomes_indices = [indice['index'] for indice in indices]
nomes_indices

['andre',
 'andre_semantic3',
 '.ds-filebeat-8.6.2-2023.06.08-000002',
 '.ds-filebeat-8.6.2-2023.08.14-000004',
 'material_index',
 '.lists-default-000001',
 '.items-default-000001',
 '.ds-filebeat-8.6.2-2023.09.13-000005',
 '.ds-filebeat-8.6.2-2023.05.09-000001',
 'index_smn_nf',
 'metrics-endpoint.metadata_current_default',
 'sinapi_index',
 'andre_americanas',
 'bruno',
 '.ds-filebeat-8.6.2-2023.07.08-000003',
 'sicro_index']

In [38]:
# import traceback
# try:
#     response = helpers.bulk(es, documentos_nf_index_generator)

#     if response[1]:
#         errors = response[1]
#         for error in errors:
#             print("Falha de indexação:", error)
# except helpers.BulkIndexError as e:
#     for err in e.errors:
#         print("Falha de indexação:", err)
#     traceback.print_exc()

# -------------------------------
# import traceback
# from elasticsearch import helpers

# try:
#     response = helpers.bulk(es, doc_generator(INDEX_NAME, df_nf_unique), chunk_size=100)
#     print('teste')  # Tente um chunk_size de 50 ou menor
#     if response[1]:
#         errors = response[1]
#         for error in errors:
#             print("Falha de indexação:", error)
# except helpers.BulkIndexError as e:
#     for err in e.errors:
#         print("Falha de indexação:", err)
#     traceback.print_exc()

# ----------------
import traceback
from elasticsearch import helpers

n = 50
for i in range(0, len(df_nf_unique), n):
    df_chunk = df_nf_unique.iloc[i:i + n]
    documentos_nf_index_generator = doc_generator(INDEX_NAME, df_chunk)
    
    try:
        response = helpers.bulk(es, documentos_nf_index_generator)
        if response[1]:
            errors = response[1]
            for error in errors:
                print("Falha de indexação:", error)
    except helpers.BulkIndexError as e:
        for err in e.errors:
            print("Falha de indexação:", err)
        traceback.print_exc()


In [45]:
query_embedding = modelL6V2.encode("açucar 1kg cristal")


query_smt= {
    'field': 'all_mini_base_vector',
    'query_vector': query_embedding.tolist(),
    'k': 10,
    'num_candidates': 50
}

results = es.search(index=INDEX_NAME, knn=query_smt, source=['descricao_produto'])

results = [
    {
        "_score": hit["_score"],
        "_source": hit["_source"]
    }
    for hit in results["hits"]["hits"]
]

for hit in results:
    print("Score:", hit["_score"])
    print("Source:", hit["_source"])


Score: 0.9964857
Source: {'descricao_produto': 'ACUCAR CRISTAL 1KG'}
Score: 0.9964857
Source: {'descricao_produto': 'ACUCAR CRISTAL  1KG'}
Score: 0.9776648
Source: {'descricao_produto': 'ACUCAR CRISTAL 2KG'}
Score: 0.96522844
Source: {'descricao_produto': 'ACUCAR CRISTAL 1X1KG'}
Score: 0.96506596
Source: {'descricao_produto': 'ACUCAR CRISTAL 1KG UNIAO'}
Score: 0.9643695
Source: {'descricao_produto': 'ACUCAR  UNIAO 1KG CRISTAL'}
Score: 0.9615716
Source: {'descricao_produto': 'ACUCAR CRISTAL 1X2KG'}
Score: 0.9597418
Source: {'descricao_produto': 'ACUCAR CRISTAL UNIAO 1KG CRISTALCUCAR'}
Score: 0.9562295
Source: {'descricao_produto': 'ACUCAR ITAMARATY CRISTAL 1KG'}
Score: 0.9545809
Source: {'descricao_produto': 'ACUCAR CRISTAL ITAMARATY 1KG'}


In [46]:
query_embedding = modelL6V2.encode("fone wireless")


query_smt= {
    'field': 'all_mini_base_vector',
    'query_vector': query_embedding.tolist(),
    'k': 10,
    'num_candidates': 50
}

results = es.search(index=INDEX_NAME, knn=query_smt, source=['codigo_barra', 'descricao_produto'])

results = [
    {
        "_score": hit["_score"],
        "_source": hit["_source"]
    }
    for hit in results["hits"]["hits"]
]

for hit in results:
    print("Score:", hit["_score"])
    print("Source:", hit["_source"])

Score: 0.8938648
Source: {'codigo_barra': 7893663400118, 'descricao_produto': 'FONE BLUETOOTH'}
Score: 0.8938648
Source: {'codigo_barra': 7898592345349, 'descricao_produto': 'Fone Bluetooth'}
Score: 0.89386475
Source: {'codigo_barra': 7896879760205, 'descricao_produto': 'Fone bluetooth'}
Score: 0.8682209
Source: {'codigo_barra': 4893948974498, 'descricao_produto': 'WIRELESS HEADSET FONE PR'}
Score: 0.8594676
Source: {'codigo_barra': 789568051739, 'descricao_produto': 'Fone Bluetooth f9-5'}
Score: 0.85610193
Source: {'codigo_barra': 7795049554401, 'descricao_produto': 'fone'}
Score: 0.85610193
Source: {'codigo_barra': 9781000152432, 'descricao_produto': 'FONE'}
Score: 0.8404286
Source: {'codigo_barra': 6920680829484, 'descricao_produto': 'FONE BLUETOOTH TWS AIRPLUS(F80)'}
Score: 0.8387755
Source: {'codigo_barra': 190199246850, 'descricao_produto': 'Fone Bluetooth TWS Pro'}
Score: 0.83475125
Source: {'codigo_barra': 6920680869206, 'descricao_produto': 'FONE BLUETOOTH TWS F90T'}


In [7]:
query = {
    "query": {
        "match": {
            "descricao_produto": "açucar 1kg cristal"
        }
    },
    "size": 16,
}

resp = es.search(index=INDEX_NAME, body=query)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]['codigo_barra'])
    print(hit["_source"]['descricao_produto'])
    print('----------------------------')
    


Got 2348 Hits:
4893948974498
WIRELESS HEADSET FONE PR
----------------------------
846885010303
Fone True Wireless Earbuds - Champion
----------------------------
7793617312415
FONE DE OUVIDO WIRELESS COM LED - VERMELHO
----------------------------
7908412811203
FONE DE OUVIDO WIRELESS HS-391 HREBOS
----------------------------
7793617303208
FONE DE OUVIDO WIRELESS COM LED - PRATA
----------------------------
6925281997020
FONE TRUE WIRELESS JBL LIVEPRO2 PRATA NA
----------------------------
4893948974481
FONE DE OUV WIRELESS EARPHONE AZUL TMV
----------------------------
7891356092541
Fone De Ouvido Philco PFO03BTR Wireless Highbeat 22H
----------------------------
7891143098404
Fone De Ouvido Wireless Bluetooth Inpods 12 Branco
----------------------------
7890000632669
Fone Ouvido Awei B925bl Bluetooth Wireless Esporte Earphone
----------------------------
6925281953521
FONE TRUE WIRELESS JBL WAVE BUDS PRETO NA
----------------------------
840063202375
Fone de Ouvido True Wireless P

In [6]:
query_fuzziness = {
    "query": {
        "match": {
            "descricao_produto": {
                "query": "feijo",
                "fuzziness": 1
            }
        }
    },
    "size": 16,
}

resp = es.search(index=INDEX_NAME, body=query_fuzziness)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]['codigo_barra'])
    print(hit["_source"]['descricao_produto'])
    print('----------------------------')
    


Got 2369 Hits:
7898014400168
CAFE FEIJO 250G
----------------------------
7896183910525
FEIJAO
----------------------------
7896725341534
POTE MANTIMENTO FEIJAO POTE MANTIMENTO FEIJAO
----------------------------
7898189491282
FEIJAO ZAIRE
----------------------------
7898935428043
FEIJAO CARIOQUINHA
----------------------------
7898941859947
BOLA FEIJAO
----------------------------
7897326100339
FEIJAO BERNADO
----------------------------
7896327514343
SOPAO FEIJAO
----------------------------
7896099100065
FEIJAO CARIOCA
----------------------------
7898189491428
FEIJAO 1KG
----------------------------
7897326300036
Feijao manteiga
----------------------------
7898276500200
FEIJAO CAUPI
----------------------------
7898014400168
CAFE FEIJO TORRADO E MOIDO
----------------------------
7897924800877
FEIJAO PRETO
----------------------------
7898189491367
FEIJÃO 1KG
----------------------------
7897326100339
FEIJAO BERNARDO
----------------------------


In [20]:
query_phrase_anzr = {
    "query": {
        "match_phrase": {
            "descricao_produto.acentuado": "açucar cristalizado"
        }
    },
    "size": 8
}
resp = es.search(index=INDEX_NAME, body=query_phrase_anzr)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]['descricao_produto'])

Got 3 Hits:
AÇUCAR CRISTALIZADO 1KG
AÇUCAR CRISTALIZADO  1KG
AÇUCAR CRISTALIZADO  ITAMARATI 1KG


In [21]:
query_filter = {
    "query": {
        "bool": {
        "must": [
            {
            "match": {
                "descricao_produto": "doce de leite"
            }
            }
        ],
        "filter": [
            {
                "range": {
                    "data_emissao": {
                        "gte": "2023-01-01 00:00:00.000", 
                        "lte": "2023-01-31 00:00:00.000"                  
                    }

                },
                
            }
        ]
    }
  },
  'size': 20,
  
}

resp = es.search(index=INDEX_NAME, body=query_filter)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"])

Got 10000 Hits:
{'modelo': 55, 'serie': 1, 'numero_nota_fiscal': 39398, 'data_emissao': '2023-01-17 00:00:00.000', 'cnpj_cpf_emitente': 24396886002308, 'razao_social_emitente': 'COMERCIAL CENTERMIX LTDA', 'tipo_destinatario': 'PESSOA_JURIDICA', 'valor_produto': 27.45, 'codigo_barra': 7898952949309, 'ncm': 21069090, 'descricao_produto': 'COBERTURA DOCE DE LEITE BRIGATTA', 'vetor_denso': [-0.012153604999184608, 0.04956986755132675, -0.036207493394613266, -0.03169063851237297, -0.040912143886089325, 0.07226831465959549, 0.08900487422943115, 0.056565407663583755, -0.040447428822517395, 0.017913250252604485, 0.07532648742198944, -0.022236211225390434, -0.021155577152967453, -0.025156058371067047, -0.065254345536232, -0.06891122460365295, -0.033592648804187775, 0.049456458538770676, -0.005152308382093906, 0.07640771567821503, 0.05217638239264488, -0.01087704673409462, -0.02681930549442768, 0.07851308584213257, -0.06549987941980362, -0.0067413607612252235, 0.04758356139063835, -0.045091524720

In [23]:
query_pesos = {
  "query": {
    "multi_match": {
      "query": "moto",
      "fields": ["descricao_produto^2", "razao_social_emitente"],
      "type": "cross_fields",
    }
  }
}

resp = es.search(index=INDEX_NAME, body=query_pesos)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]["descricao_produto"])
    print(hit["_source"]["razao_social_emitente"])
    print("__________________")


Got 8439 Hits:
MOTO FIRENZE
BS TOYS INDUSTRIA E COMERCIO DE BRINQUEDOS LTDA
__________________
MOTO SPYRIT
USUAL BRINQUEDOS LTDA
__________________
MOTO DUO
CALESITA INDUSTRIA DE     BRINQUEDOS LTDA
__________________
MOTO UNO
CALESITA INDUSTRIA DE     BRINQUEDOS LTDA
__________________
MOTO RACER
LIDER IND. E COM. DE  BRINQUEDOS LTDA
__________________
MOTO SNAKE
USUAL PLASTIC IND. E COM. DE ART. PLASTICOS LTDA
__________________
MOTO TRILHA
VANTAJ ALBATROZ DISTRI DE BRIN E UTILIDADES DOMESTICAS LTDA.
__________________
MOTO MULTISTRADA
AMAR E...IND E COM DE BRINQ LTDA
__________________
MOTO DAYTONA
AMAR E...IND E COM DE BRINQ LTDA
__________________
Mini-Moto
MINI PLAY IND E COM DE PLASTICOS LTDA
__________________


In [25]:
query_pesos_diff = {
  "query": {
    "multi_match": {
      "query": "moto",
      "fields": ["descricao_produto", "razao_social_emitente^2"],
      "type": "cross_fields",
    }
  }
}

resp = es.search(index=INDEX_NAME, body=query_pesos_diff)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]["descricao_produto"])
    print(hit["_source"]["razao_social_emitente"])
    print("__________________")


Got 8439 Hits:
RETENTOR BENGALA CORTECO YAMAHA YBR 125
TOINHO MOTO PECAS
__________________
Bico Frontal Cromo Forte Biz + 125 Ks - Es - Ex Branco 2011 - 2016
NACAO MOTO PARTS
__________________
Rabeta Traseira Pro Tork Nxr 160 Bros 2015 - 2016 Preto
RACE MOTO PECAS
__________________
Tampa Lateral Pro Tork Nxr Bros 160 2015 - 2017 Preto
RACE MOTO PECAS
__________________
Capacete Fechado Pro Tork V-pro Jet 3 Preto - Azul Vis. Fume Tam. 58
JAPINHA MOTO POWER
__________________
Rabeta Completa Pro Tork Ybr 125 2006 - 2007 Verde
WRD MOTO PECAS
__________________
Paralama Dianteiro Pro Tork Fazer 150 2014 Azul
RACE MOTO PECAS
__________________
Rabeta Central Pro Tork Fazer 150 2014 Preto
RACE MOTO PECAS
__________________
Protetor Espuma Guidao Fat Bar X-cell ProTaper - Vermelho / Azul
Celerion Moto Pecas
__________________
Esticador Tensor Da Corrente De Transmissao Xtz 250 Lander
MOTO CINTRA LTDA
__________________


In [26]:
query_must = {
  "query": {
    "bool": {
            "must": [
                {"match": {"descricao_produto": "queijo"}},
            ],
            "must_not": [
                {"match": {"descricao_produto": "ralado"}}
            ],
    }
  },"size": 20
}
resp = es.search(index=INDEX_NAME, body=query_must)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]['descricao_produto'])


Got 3972 Hits:
QUEIJO
KRO QUEIJO SABOR QUEIJO 100G
QUEIJO MUSSARELA CASA QUEIJO 400G FAT.
SALG.BEBELA QUEIJO
SALG.TIME QUEIJO
SALG.LUKITAO QUEIJO
QUEIJO COALHO
SALG.KRO QUEIJO
SALG.AMAVITA QUEIJO
SALG.MIKITOS QUEIJO
KRO QUEIJO
QUEIJO PRATO
QUEIJO MUSSARELA
AMENDUPA QUEIJO
SALG.YOKITOS QUEIJO
QUEIJO COLONIAL
QUEIJO; MUSSARELA
PRENSA QUEIJO
QUEIJO PARMESAO - FRACIONADO
QUEIJO GORGONZOLA - FRACIONADO


In [27]:
queryShould = {
  "query": {
    "bool": {
      "should": [
        { "match": { "descricao_produto": "farofa" }},
        { "match": { "descricao_produto": "bacon" }}
      ],
    }
  },"size": 26
}

resp = es.search(index=INDEX_NAME, body=queryShould)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]['descricao_produto'])

Got 1025 Hits:
FAROFA MAND ACRE FAROFA 500G BACON
FAROFA DE BACON 500g
FAROFA APTI BACON 250GR
FAROFA MIKA BACON 250G
FAROFA GOURMET 500G BACON
FAROFA GOURMET BACON KAMAR 280G
FAROFA PRONTA BACON SINHA 10X250G
FAROFA CASEIRA BACON 400GR 3730
Farofa Pronta Sinha 250g Bacon
FAROFA PRONTA SINHA 250G BACON
FAROFA PRONTA POLETTO 300G BACON
FAROFA CASEIRA BACON 300 G
FAROFA CASEIRA BACON POLETTO 300 GR
FAROFA DU ACRE GOURMET 500G BACON
FAROFA MAND ACRE FAROFA 500G BACON UN QTD. 1.00 UN
POTE PARA FAROFA       POTE PARA FAROFA
FAROFA MAND ACRE FAROFA 500G ALHO
POTE PARA FAROFA
FAROFA DOCE VIAPANE
FAROFA ZAELI ORIGINAL
FAROFA PICANTE 12X250G
FAROFA TRADICIONAL 12X250G
FAROFA DOCE 5KG
SALG.CRANT BACON
SALG.MIKITOS BACON
BACON EXCELENCIA


In [ ]:
# Oobter os embeddings de um texto
def get_embeddings(text):
    return modelL6V2.encode(text)

# Consulta semântica
def semantic_search(query_text, index_name='index_s'):
    query_embeddings = get_embeddings(query_text)
    script_query = {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                "params": {"query_vector": query_embeddings.tolist()}
            }
        }
    }

    response = es.search(
        index=index_name,
        body={
            "size": 10,
            "query": script_query,
            "_source": {"includes": ["title", "content"]}
        }
    )
    return response['hits']['hits']

query = "jeans"
results = semantic_search(query)
for hit in results:
    print(hit["_source"])